 # 1.加载模型

In [1]:
from transformers import BertTokenizer

In [2]:
model_name = 'bert-base-uncased'

In [3]:
tokenizer = BertTokenizer.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

C:\Users\juju\AppData\Roaming\Python\Python312\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\juju\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
tokenizer
#vocab_size 30522: token >ids 映射
#最大长度512
#右侧添加padding
#从右侧截断
#special token map

BertTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [5]:
special_tokens = list(tokenizer.special_tokens_map.values())

In [ ]:
special_tokens
#sep 句尾 CLS 句首

['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']

In [7]:
tokenizer.convert_tokens_to_ids(special_tokens)

[100, 102, 0, 101, 103]

In [8]:
tokenizer.encode(special_tokens)

[[101, 100, 102],
 [101, 102, 102],
 [101, 0, 102],
 [101, 101, 102],
 [101, 103, 102]]

# 2认识文本语料

- newsgroups_train.DESCR
- newsgroups_train.data
- newsgroups_train.target
- newsgroups_train.target_names 

In [9]:
from sklearn.datasets import fetch_20newsgroups

In [10]:
newsgroup_train = fetch_20newsgroups(subset='train')

In [12]:
type(newsgroup_train.data)

list

In [13]:
len(newsgroup_train.data)

11314

In [15]:
len(newsgroup_train.target)

11314

In [16]:
from collections import Counter

In [18]:
Counter(newsgroup_train.target)

Counter({10: 600,
         15: 599,
         8: 598,
         9: 597,
         11: 595,
         7: 594,
         13: 594,
         14: 593,
         5: 593,
         2: 591,
         12: 591,
         3: 590,
         6: 585,
         1: 584,
         4: 578,
         17: 564,
         16: 546,
         0: 480,
         18: 465,
         19: 377})

In [20]:
newsgroup_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

# 3tokenizer 补充

- input_ids，attention_mask 
    - mask:bert 另一个与训练任务， mlm，对word 添加掩码，添加之后mask即为1
- encode_plus 已经消失
- token_type_ids
    - token_type：0 > 第一句、例句；1 > 第二句。可以通过 tokenizer() (tokenizer.\_\_call\_\_：所有句子都为0)或者通过tokenizer.encode生成、返回，前一句被设置为0后一句设置为1。
    - 句子对一般使用在 nsp （next sentence predict）下一句子预测任务上，bert的预训练任务
    

In [21]:
test_news = newsgroup_train.data[:3]

In [ ]:
#len(test_news[0])
list(map(len, test_news))

[721, 858, 1981]

In [ ]:
tokenizer(test_news,max_length=32,truncation=True)
#句子级别，不指定的情况下 token_type_ids均为0

{'input_ids': [[101, 2013, 1024, 3393, 2099, 2595, 3367, 1030, 11333, 2213, 1012, 8529, 2094, 1012, 3968, 2226, 1006, 2073, 1005, 1055, 2026, 2518, 1007, 3395, 1024, 2054, 2482, 2003, 2023, 999, 1029, 102], [101, 2013, 1024, 3124, 5283, 2080, 1030, 9806, 1012, 1057, 1012, 2899, 1012, 3968, 2226, 1006, 3124, 13970, 2080, 1007, 3395, 1024, 9033, 5119, 8554, 1011, 2345, 2655, 12654, 1024, 2345, 102], [101, 2013, 1024, 1056, 29602, 6856, 1030, 14925, 1012, 14925, 2078, 1012, 19749, 1012, 3968, 2226, 1006, 2726, 1041, 12688, 1007, 3395, 1024, 1052, 2497, 3980, 1012, 1012, 1012, 3029, 1024, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [36]:
print(tokenizer(test_news, max_length=32,truncation=True).keys())

KeysView({'input_ids': [[101, 2013, 1024, 3393, 2099, 2595, 3367, 1030, 11333, 2213, 1012, 8529, 2094, 1012, 3968, 2226, 1006, 2073, 1005, 1055, 2026, 2518, 1007, 3395, 1024, 2054, 2482, 2003, 2023, 999, 1029, 102], [101, 2013, 1024, 3124, 5283, 2080, 1030, 9806, 1012, 1057, 1012, 2899, 1012, 3968, 2226, 1006, 3124, 13970, 2080, 1007, 3395, 1024, 9033, 5119, 8554, 1011, 2345, 2655, 12654, 1024, 2345, 102], [101, 2013, 1024, 1056, 29602, 6856, 1030, 14925, 1012, 14925, 2078, 1012, 19749, 1012, 3968, 2226, 1006, 2726, 1041, 12688, 1007, 3395, 1024, 1052, 2497, 3980, 1012, 1012, 1012, 3029, 1024, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [37]:
tokenizer.encode??

Signature:
tokenizer.encode(
    text: 'TextInput | PreTokenizedInput | EncodedInput',
    text_pair: 'TextInput | PreTokenizedInput | EncodedInput | None' = None,
    add_special_tokens: 'bool' = True,
    padding: 'bool | str | PaddingStrategy' = False,
    truncation: 'bool | str | TruncationStrategy | None' = None,
    max_length: 'int | None' = None,
    stride: 'int' = 0,
    padding_side: 'str | None' = None,
    return_tensors: 'str | TensorType | None' = None,
    **kwargs,
) -> 'list[int]'
Docstring:
Converts a string to a sequence of ids (integer), using the tokenizer and vocabulary.

Same as doing `self.convert_tokens_to_ids(self.tokenize(text))`.

Args:
    text (`str`, `list[str]` or `list[int]`):
        The first sequence to be encoded. This can be a string, a list of strings (tokenized string using the
        `tokenize` method) or a list of integers (tokenized string ids using the `convert_tokens_to_ids`
        method).
    text_pair (`str`, `list[str]` or `list[int]

In [39]:
tokenizer.encode(text=test_news[0],text_pair=test_news[1],max_length=32,truncation=True)

[101,
 2013,
 1024,
 3393,
 2099,
 2595,
 3367,
 1030,
 11333,
 2213,
 1012,
 8529,
 2094,
 1012,
 3968,
 102,
 2013,
 1024,
 3124,
 5283,
 2080,
 1030,
 9806,
 1012,
 1057,
 1012,
 2899,
 1012,
 3968,
 2226,
 1006,
 102]

In [42]:
tokenizer.encode_plus(text=test_news[0], text_pair=test_news[1],max_length=32,truncation=True).token_type_ids

AttributeError: BertTokenizer has no attribute encode_plus

In [43]:
tokenizer.decode(tokenizer.encode(text=test_news[0], text_pair=test_news[1],max_length=32,truncation=True))

'[CLS] from : lerxst @ wam. umd. ed [SEP] from : guykuo @ carson. u. washington. edu ( [SEP]'

---

## 知识点总结

### 1. Transformer Tokenizer 方法对比

| 方法 | 返回类型 | 用途 | 状态 |
|------|----------|------|------|
| `tokenizer()` / `tokenizer.__call__()` | `dict` | 日常使用，获取完整信息 | ✅ 推荐 |
| `tokenizer.encode()` | `list` | 快速获取 token IDs | ✅ 可用 |
| `tokenizer.encode_plus()` | `dict` | 获取完整信息 | ❌ 已废弃 |

**示例对比：**
```python
# tokenizer() - 返回完整字典
output = tokenizer(text, text_pair=text2, max_length=32, truncation=True)
# output.keys() → ['input_ids', 'token_type_ids', 'attention_mask']

# encode() - 只返回 input_ids 列表
ids = tokenizer.encode(text, text_pair=text2, max_length=32, truncation=True)
# ids → [101, 2013, 1024, ...]
```

**设计理念：**
- `tokenizer()`: 高级 API，返回模型需要的所有信息
- `encode()`: 简化 API，专注核心功能（文本 → ID）

---

### 2. Python 3 的 `dict.keys()` 返回 KeysView

```python
d = {'input_ids': [1, 2, 3], 'token_type_ids': [0, 0, 0]}
keys = d.keys()
print(type(keys))  # <class 'dict_keys'> (KeysView)
```

**KeysView 特性：**
- ✅ **动态视图**：反映字典的实时状态
- ✅ **只包含键**：不包含值，支持 `len()`, `in`, 迭代
- ✅ **内存高效**：不复制数据，只是引用
- ⚠️ **Jupyter 显示**：会显示完整字典内容便于调试

**验证：**
```python
keys = d.keys()
print(list(keys))  # ['input_ids', 'token_type_ids']
d['new_key'] = 'value'
print(list(keys))  # ['input_ids', 'token_type_ids', 'new_key']  # 视图自动更新
```

**如果你只想要键的列表：**
```python
keys_list = list(tokenizer(...).keys())
```

---

### 3. Tokenizer 输出字段说明

| 字段 | 含义 | 说明 |
|------|------|------|
| `input_ids` | Token ID 序列 | 文本转换成的数字索引 |
| `token_type_ids` | 句子类型标识 | 0=第一句，1=第二句（用于 NSP 任务） |
| `attention_mask` | 注意力掩码 | 1=真实 token，0=padding token |

**注意：**
- 单句子输入时，`token_type_ids` 全部为 0
- 句子对输入时，第一句为 0，第二句为 1
- 使用 `tokenizer()` 时默认所有句子为 0
- 使用 `encode()` 指定 `text_pair` 时会自动设置 `token_type_ids`

---

### 4. 迁移指南（旧代码 → 新代码）

```python
# ❌ 旧代码（不再工作）
output = tokenizer.encode_plus(text, text_pair=text2)
ids = output['input_ids']

# ✅ 新代码
output = tokenizer(text, text_pair=text2)  # 推荐
ids = output.input_ids

# 或
ids = tokenizer.encode(text, text_pair=text2)  # 简单场景
```

---

## 📝 2022 年教程 → 2025 年更新要点

### 已废弃的 API
- ❌ `tokenizer.encode_plus()` → ✅ 使用 `tokenizer()`
- ❌ 直接访问返回字典的 `.token_type_ids` 属性在 `encode_plus` 中失效

### 新增推荐参数

#### 1. `return_tensors` - 返回框架张量
```python
# 返回 PyTorch 张量
output = tokenizer(text, return_tensors='pt')
# output.input_ids 是 torch.Tensor

# 返回 TensorFlow 张量
output = tokenizer(text, return_tensors='tf')
# 返回 NumPy 数组
output = tokenizer(text, return_tensors='np')
```

#### 2. `padding` - 填充策略
```python
# 动态填充到批次中最长序列
tokenizer(batch_texts, padding=True)

# 填充到指定长度
tokenizer(batch_texts, padding='max_length', max_length=128)

# 不填充（默认）
tokenizer(batch_texts, padding=False)
```

#### 3. `truncation` - 截断策略
```python
# 截断到模型最大长度
tokenizer(long_text, truncation=True)

# 截断到指定长度
tokenizer(long_text, truncation=True, max_length=128)

# 只截断第一句
tokenizer(text, text_pair=text2, truncation='only_first')
```

#### 4. `add_special_tokens` - 控制特殊 token
```python
# 默认添加 [CLS] 和 [SEP]
tokenizer(text)  # 添加特殊 tokens

# 不添加特殊 tokens
tokenizer(text, add_special_tokens=False)
```

### 批处理最佳实践

```python
# ✅ 推荐：自动处理 padding 和 truncation
batch_output = tokenizer(
    ['text 1', 'text 2', 'text 3'],
    padding=True,          # 自动填充到批次最长
    truncation=True,       # 自动截断超长文本
    max_length=512,        # 最大长度
    return_tensors='pt'    # 返回 PyTorch 张量
)

# 访问结果
input_ids = batch_output.input_ids          # shape: [3, max_len]
attention_mask = batch_output.attention_mask
```

### 字典访问方式（两种都支持）

```python
result = tokenizer(text)

# 方式 1：点号访问（推荐，更简洁）
ids = result.input_ids
mask = result.attention_mask

# 方式 2：字典访问（兼容性强）
ids = result['input_ids']
mask = result['attention_mask']
```

### 常用参数组合速查

```python
# 🔥 日常使用（推荐）
tokenizer(text, padding='longest', return_tensors='pt')

# 🔥 单条文本，不填充
tokenizer(text, truncation=True)

# 🔥 批量处理，固定长度
tokenizer(batch_texts, padding=True, truncation=True, max_length=128, return_tensors='pt')

# 🔥 句子对（如问答任务）
tokenizer(question, text_pair=passage, padding=True, truncation=True)

# 🔥 不添加特殊 token（某些特殊场景）
tokenizer(text, add_special_tokens=False)
```

### 注意事项

1. **版本兼容性**：2022 年教程中的 `encode_plus` 在 2025+ 版本中已完全移除
2. **默认行为变化**：新版本默认不启用 padding，需要显式指定 `padding=True`
3. **性能优化**：批处理时推荐一次性传入列表，而不是循环调用
4. **框架集成**：使用 `return_tensors='pt'` 可直接返回模型输入格式

### 推荐学习路径

```python
# 1️⃣ 基础使用
tokenizer(text)

# 2️⃣ 添加长度控制
tokenizer(text, truncation=True, max_length=512)

# 3️⃣ 批处理
tokenizer(batch_texts, padding=True, truncation=True)

# 4️⃣ 模型就绪格式
tokenizer(batch_texts, padding=True, truncation=True, return_tensors='pt')

# 5️⃣ 完整流程
inputs = tokenizer(text, return_tensors='pt')
outputs = model(**inputs)
```